# Phase Features with ridge regression

In this tutorial, we will demonstrate the use of phase features in distribution regression (for the aerosol dataset), in comparison with the normal Gaussian Kernel (Fourier Features approach), and in a scenario of covariate shift. Parameters were chosen previosuly with cross validation.

For this tutorial, we will make use of the kerpy package from https://github.com/oxmlcs/kerpy.

This is tested on Python 2.7.

In [1]:
from __future__ import print_function

import numpy as np
from kerpy.GaussianKernel import GaussianKernel
from kerpy.LinearKernel import LinearKernel
from kerpy.LinearBagKernel import LinearBagKernel

from SymInvBagKernel import SymInvBagKernel
import aux_fct

Specify path for data and load the aerosol dataset (MISR1).

In [2]:
np.random.seed(23) # Set seed
path = '/Users/Leon1/Desktop/Fourier-Phase-Neural-Network' #Change path 
misr_data_x, misr_data_y = aux_fct.load_data(path, random = True) 
print('Data shape:',misr_data_x.shape)
# Train Set
train_x = misr_data_x[:640]
train_y = misr_data_y[:640]
# Test Set
test_x = misr_data_x[640:]
test_y = misr_data_y[640:]
variance = np.var(np.concatenate(misr_data_x), axis = 0) # For calculating signal to noise ratio
data_dim = misr_data_x.shape[2]
mean_predict_t = np.linalg.norm(test_y - np.mean(train_y))**2/len(test_y) #Mean prediction

Data shape: (800, 100, 16)


Calculate the bandwidth for phase features kernel (note that a larger scale will give you a more robust model, so when tuning, one should choose the larger scale within an acceptable range of error):

In [3]:
bandwidth_scale = 1.25
bandwidth = np.sqrt(aux_fct.median_sqdist(train_x) / 2) * bandwidth_scale # Calculate bandwidth
print('Bandwidth:', bandwidth)

Bandwidth: 370.057460792


Setup the kernel using the kerpy kernel class structure:

In [4]:
data_phase_kernel = GaussianKernel(sigma=float(bandwidth))
phase_kernel = SymInvBagKernel(data_phase_kernel) #Defines the phase features kernel, normalising the fourier features
phase_kernel.rff_generate(mdata= 250, dim= data_dim) #Generate the frequencies from a normal distribution using the bandwidth.

Construct explicit feature maps for bags. 

In [5]:
train_phase_means = phase_kernel.rff_expand(train_x)
test_phase_means = phase_kernel.rff_expand(test_x)

By constructing an explicit feature map for each bag as below, we can now use any normal regression methods on $\mathbb{R}^{16}$, here we use ridge regression (with a linear kernel).

In [6]:
lmbda = 0.01 #Ridge regularisation parameter
ridge_kernel = LinearKernel()
obj_phase, t_predict, t_phase_mse = ridge_kernel.ridge_regress(train_phase_means, train_y, lmbda, test_phase_means, test_y)
print('Mean RMSE:', np.sqrt(mean_predict_t))
print('Phase RMSE:', np.sqrt(t_phase_mse))

Mean RMSE: 0.162785267464
Phase RMSE: 0.0711980953622


Now we do the same for the Gaussian Kernel. Note the LinearKernel() of the data_kernel provides 

In [7]:
#This computes the mean embedding (average) of the explicit feature map of the data_kernel (Gaussian), 
#effectively providing us a gaussian bag kernel. 
data_gauss_kernel = GaussianKernel(sigma=float(bandwidth))
gauss_kernel = LinearBagKernel(data_gauss_kernel)
gauss_kernel.rff_generate(mdata= 250, dim= data_dim)
train_gauss_means = gauss_kernel.rff_expand(train_x)
test_gauss_means = gauss_kernel.rff_expand(test_x)

lmbda = 0.001 #Ridge regularisation parameter
ridge_kernel = LinearKernel()
obj_gauss, t_predict, t_gauss_mse = ridge_kernel.ridge_regress(train_gauss_means, train_y, lmbda, test_gauss_means, test_y)
print('Mean RMSE:', np.sqrt(mean_predict_t))
print('Fourier (Gaussian) RMSE:', np.sqrt(t_gauss_mse))


Mean RMSE: 0.162785267464
Fourier (Gaussian) RMSE: 0.0716890964235


We see that the Fourier features (Gaussian Kernel on Bags) and Phase Features perfrom similarly in this case. Now we demonstrate that phase features are invariant up to Symmetric Positive Definite (SPD) noise, by artficially creating a scenario of covariate shift, where we add __only__ noise to the test set.  

In [8]:
noise_ratio_t = 1.0 # This is a super high noise to signal setting.
test_x_noisy = np.zeros((160, 100, 16))
latent_t = noise_ratio_t * variance * np.random.uniform(low = 0.0, high = 1.0, size = (160, 1, 16))
for i in range(160):
    for k in range(16):
        test_x_noisy[i,:,k] = test_x[i,:,k] + np.sqrt(latent_t[i,:,k]) * np.random.normal(size = (100))

Observe that the features are very different (for the first sample in first bag).

In [9]:
print('Original: First sample in first bag\n',test_x[0][0])
print('Noisy: First sample in first bag\n', test_x_noisy[0][0])

Original: First sample in first bag
 [ 372.33903211  273.97752857  222.03223966  206.53149968  347.60454266
  265.22960477  237.22911295  193.27649299  421.43038523  274.16365461
  232.19638219  194.04713291   21.39239526   38.21109247  116.34367275
  130.11828423]
Noisy: First sample in first bag
 [ 360.92110067  306.48349261   88.3272009   243.34740126  449.54540514
  231.61697426  253.41464995  233.29682893  394.25861008  260.39300882
  296.05358639  225.80549346   24.2055456    22.41574521  127.6613082
  122.33246268]


Obtain RMSE on the noisy test set using non-noisy training set.

In [11]:
test_noisy_gauss_means = gauss_kernel.rff_expand(test_x_noisy)
test_noisy_phase_means = phase_kernel.rff_expand(test_x_noisy)

ridge_kernel = LinearKernel()
t_noisy_phase_predict = np.dot(obj_phase.T,ridge_kernel.kernel(train_phase_means,test_noisy_phase_means))
t_noisy_gauss_predict = np.dot(obj_gauss.T,ridge_kernel.kernel(train_gauss_means,test_noisy_gauss_means))

print('Mean RMSE:', np.sqrt(mean_predict_t))
print('\nPHASE')
print('Phase RMSE:', np.sqrt(t_phase_mse))
print('Noisy Phase RMSE:', np.sqrt((np.linalg.norm(t_noisy_phase_predict.T - test_y)**2)/len(test_y)))
print('\nFOURIER')
print('Fourier (Gaussian) RMSE:', np.sqrt(t_gauss_mse))
print('Noisy Fourier (Gaussian) RMSE:', np.sqrt((np.linalg.norm(t_noisy_gauss_predict.T - test_y)**2)/len(test_y)))


Mean RMSE: 0.162785267464

PHASE
Phase RMSE: 0.0711980953622
Noisy Phase RMSE: 0.0961964367606

FOURIER
Fourier (Gaussian) RMSE: 0.0716890964235
Noisy Fourier (Gaussian) RMSE: 0.132692527392


We see that indeed the Phase features, which are invariant to SPD noise (Gaussian Noise in this case) performs much better.